# Generative AI - Combient Mix AB / Silo AI for Patricia AB 2023

Prompt Engineering (PE) is the primary vehicle for guiding generative AI models towards stable applications. It is particularly applicable to Large Language Models (LLMs) and involves formulating prompts and prompting schemas in order to retrieve appropriate responses to queries. It is essential for meaningful interactions with LLMs.

This notebook, which is the second of two notebooks, provides a hands-on introduction to PE for end users with a basic understanding of the Python programming language. No advanced coding or technical background knowledge of generative AI or LLM is required for completing the exercises in the notebook. The notebook is divided into three sections, each corresponding to material covered in two live seminars. There will be ~60 minutes available in order to go through and complete exercise 3.
<br />
<br />

***

**Structure**

Exercise **1 & 2 during the first session** (~ 40 minutes per exercise) and exercise **3 during the second session** (~ 60 minutes)


1.) First, we'll explore the basics of prompting LLMs and the importance of PE. We will see examples of some important and cutting edge techniques for prompting efficiently and with intent
* 0 / 1 / few-shot prompting,
* Role-Task-Format (RTF),
* In-Context-Learning (ICL),
* Chain-of-Thought (CoT) and
* Tree-of-Thought (ToT)

2.) We'll learn how to integrate and utilize existing tools for specific tasks. We focus on some of the tools available via enabling plugins for GPT-4 at [OpenAI chat interface](https://chat.openai.com/). Example tasks include
  * **Searching the internet for recent information** LLMs are generally pre-trained and thus have a knowledge cutoff at the time when training stopped. The latest GPT-4 model of OpenAI for example has a knowledge cutoff at January 2022. This means that it was not trained on any data produced after that time, so if we want to include updated information this must be done by fetching the information and providing it as context together with our query.
  * **Using your own PDF documents as a knowledge base** (plugin: AskYourPDF). There are several plugins available for GPT-4 which provide the capability of uploading a PDF document and use it as a reference for interacting with the LLM.

3.) The third exercise is more programming heavy and involves working with API calls. We'll guide you through it! This allows us to perform more complex tasks and write our own custom functions utilizing the power of the LLM. We introduce the Python library [Langchain](https://python.langchain.com/docs/get_started/introduction.html) and some of its tools for manipulating prompts. This allows for more intricate problem-solving as well as gaining control over the reliability of the output. We will see how to use this in order to accomplish Retrieval Augmented Generation (RAG) from our own knowledge base (the PDF document). This exercise demonstrates how the plugins used in exercise 2 work under the hood. Examples of what we can achieve include

* constraining the system behaviour, for example to mitigate hallucinations from the LLM
* Retrieval Augmented Generation from an external knowledge base such as the internet or a collection of documents

This provides a deeper understanding of the fundamental components essential for constructing advanced generative AI systems.

***

**Note:**

In order to run this notebook properly you will need

* **Gmail account** - Follow the provided instructions to download the notebook to your GDrive, so that you can edit and save it freely.
* **Google GDrive access** - enable the app `Google Colaboratory`. The process is highlighted in the accompanying reading material, but you can also ask ChatGPT a question like this by posing the following query <span style="color:blue;">how do I enable google colab for the first time?</span>

> Run the below code blocks to install necessary packages.
>
> Notebook code blocks can be executed via either:
> * **shift + enter**: executes current code block and moves to the cell below
> * **control + enter**: executes current code block

## Environment setup

Here we set up the environment and make sure we can access data via Google Drive.

Run the code below to load the GenAI map we will be using during this course. The code is fetching a map called GenAI from our Github repository. Click on plus next to the cell to run the code.

In [6]:
use_drive = True

if use_drive:
  from google.colab import drive
  drive.mount('/content/drive')
  %cd /content/drive/MyDrive

  !git clone https://github.com/statisticalmodel/GenAI

Mounted at /content/drive
/content/drive/MyDrive
fatal: destination path 'GenAI' already exists and is not an empty directory.


You can now view the files by clicking on the map and follow the path down in the GenAI short course.

![Image of GitClone](https://github.com/statisticalmodel/GenAI/blob/main/GenAI%20short%20course/GitClone.png?raw=true)

## Packages & Imports

Installing and importing necessary packages/modules. Note that these should be installed only in a virtual environment when using the Colab Notebook.

Run the below code block to install some of the Python libraries which are required for running the Notebook.

Note that this may take up to ~15 seconds to complete.

> **NB: All packages below are not necessary and should be cleaned after completion of exercises**

In [7]:
!pip install -qU transformers \
    --upgrade huggingface_hub \
    -q sentencepiece \
    -q tiktoken \
    -q openai \
    -q langchain \
    -q sentence-transformers \
    -q jq \
    -q faiss-cpu \
    -q pypdf \
    -q wikipedia \
    -q duckduckgo-search \
    -q colorama \
    -q PyMuPDF

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [8]:
!pip install -qU openai==0.28.1 \
    -q langchain==0.0.330 \
    -q faiss-cpu \
    -q wikipedia \
    -q duckduckgo-search \
    -q colorama

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


Run the below code block to import the necessary library modules used in the notebook.

Note that this may take up to ~20 seconds to complete.

> **NB: All packages below are not necessaty and should be cleaned after completion of exercises**

In [9]:
# Some system and base modules
import os
import sys
from timeit import default_timer as timer
import getpass

# NLP modules
import openai
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI

# Other modules
from colorama import Fore, Back, Style

## Import helper functions used in the Notebook

Run the code block below to import customized helper functions used in the notebook. These functions have been written to, for example, process PDF files and reduce clutter in the notebook by using explicit code. All of the imported functions are located in the file `helper_functions.py` where you may explore their definitions.

In [10]:
import sys
sys.path.append('/content/drive/MyDrive/GenAI/GenAI short course/')

from helper_functions import *

## Setting the access key for OpenAI API

**The OpenAI API key can be set manually in the notebook by running the code cell block below.**

You can optionally set it as an environment variable; by typing the following in your Mac terminal
```
export OPENAI_API_KEY=sk-...
```
or if you are using windows 10 you can type the following in a Command window
```
set OPENAI_API_KEY=sk-...
```
Observe the lack of space in the value designations.

> **NB: Delete the API access key below before pushing to repo etc.**

In [11]:
# sk-...

# Here we can set the OpenAI API access key manually in case it fails to load from the environment.
if not os.environ.get("OPENAI_API_KEY"):
    api_key = getpass.getpass("Enter OpenAI API Key here")
    os.environ["OPENAI_API_KEY"] = api_key
else:
  print(f"OPENAI_API_KEY fetched from environment!")


#print(f"The Open AI access key is given by: \n\n {os.environ['OPENAI_API_KEY']}")

Enter OpenAI API Key here··········


# Hands-On 3: Customized Tools for Extended LLM Functionality

## Introduction to Interacting with ChatGPT via the OpenAI API

Example themes for exercises:

* Search the internet using Wikipedia and DuckDuckGo.
  - Useful and easy to implement and understand at a basic level.
* Search a knowledge base, e.g. contained within a PDF.
  - Useful for many purposes. Builds on previous exercises and showcase the behind the scenes work involved in the plugins in the OpenAI browser environment.

**Why should you use Python and API?**

Using the OpenAI API (Application Program Interface) to access their models, instead of the online chat interface, gives us several advantages, especially when we want to build custom made solutions. One of the big perks of using the API is automation. Python, as a common programming langauge, is great at taking care of repeating or tricky tasks for us, which is helpful when we have a lot of such tasks to handle.

With the help of python we can also get the OpenAI models to work well with other tools and software we have. Plus, it lets us adjust things to work the way we want, making our interaction with the OpenAI models more customs made.

Python has a lot of advanced features which allow us to use the OpenAI services to their full potential. This means we can do a wider variety of tasks. In this part of the course, we will look into how a model can get access to more recent data via accessing the internet. We will also see how you can extract knowledge from PDF files.

Additionally, using python helps us keep track of any changes made along the way, which is great for collaborative team work. Python also ensures that every time we run a task, it gives us consistent results, making it easier to check if something goes wrong and fix it. This reliability helps make our solutions stronger and more dependable.

We will be making use of the code library called `langchain` to call on the OpenAI models. It is a simple and relatively user-friendly way to interact with the model, making it a great choice for beginners or for those looking to get things done quickly. This method provides us access to a `temperature` parameter, which determines the randomness or stochastic nature of the output the model will give us. A temperature of 1 makes the model more random, while a temperature close to 0 makes the model more deterministic with the same output for repeated queries. Most people prefer the output to be somewhat creative in nature with a temperature value of around 0.7, which is the default value set for the OpenAI models. You can try changing this value and experience how the output changes for the same query.

In the code snippets below we call on the API. Model is specified via the `model` parameter on the second row. A complete list of available models to choose from can be found [here](https://platform.openai.com/docs/models/overview). We will start by using one of the GPT-3.5 turbo models.

* **gpt-3.5-turbo** uses the latest available model version of GPT-3.5. Allows for up to **4,097 tokens** as input and output
    * **gpt-3.5-turbo-0613** is the latest version of GPT-3.5, currently the same as above
    * **gpt-3.5-turbo-16k** is a version of GPT-3.5 with longer context window, up to **16,385 tokens**

</b>

* **gpt-4** uses the latest available model version of GPT-4. Allows using **8,192 tokens** as input and output
    * **gpt-4-0613** is the latest version of GPT-4, same as above

In [12]:
# This creates an instance of the model interface which we can subsequently call on
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    # The below parameters can be changed
    model="gpt-3.5-turbo-0613",
    temperature=0.7
)

> **NB 1: The GPT-3.5 model usually takes between 15 to 45 seconds to respond. If it takes more than a minute, stop the execution and rerun the cell block.**

> **NB 2: This is one option for implementing the LLM call, which is nice due to its simple and intuitive format. At the bottom of this section we explore using the OpenAI API directly as an alternative option, which provides even greater control of the input/output.**

The `chat_model` instance accepts a list of `messages` as input. We specify messages according to a schema used by various functions in the `langchain` library. This will facilitate some of the more advanced functionalities we will make use of later. Messages in a chat conversation are generally provided as a `list` (inside square brackets) of conversation turns.

In [13]:
from langchain.schema import (
    # Overall system behaviour of the chat bot
    SystemMessage,
    # User message, query to chat bot
    HumanMessage,
    # The response from the chat bot, for conversational turns
    AIMessage
)


# The system prompt will be placed at the top of every message and should set overall system behaviour
system_prompt = "You are a helpful assistant."

# The user prompt is what would be written in the chat interface, your query
user_prompt = "What do you know about Patricia Industries from Sweden?"


# These are collected into a messages list of
#
#   SystemMessage - the system prompt
#   HumanMessage  - the user query
#   AIMessage     - the bot response, in case you wish to continue on a conversation
messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_prompt),
]

In [14]:
# We can check what the messages look like by printing it out
messages

[SystemMessage(content='You are a helpful assistant.'),
 HumanMessage(content='What do you know about Patricia Industries from Sweden?')]

As we see, the `messages` variable is a list containing our `SystemMessage` and `HumanMessage`.


Once we have a list of messages in the above format it is easy to call the OpenAI model and get a response

In [15]:
# Here we collect the output from the chat model in the variable response
response = chat_model(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

Patricia Industries is a Swedish investment firm that is part of the Investor AB group, one of the largest industrial holding companies in Northern Europe. It was founded in 2006 and is headquartered in Stockholm, Sweden. 

Patricia Industries primarily focuses on long-term investments in companies with strong market positions and potential for sustainable value creation. They invest in a wide range of sectors, including healthcare, industrials, technology, and consumer goods. 

The firm aims to support the growth and development of its portfolio companies by providing strategic guidance, operational expertise, and financial resources. They take an active ownership approach and work closely with the management teams of their investments to drive long-term value.

Some notable investments by Patricia Industries include Mölnlycke Health Care, Permobil, Aleris, Grand Group, and Laborie. They have a global presence and invest in companies both in Sweden and internationally.

Overall, Patri

The full response is actually formatted as a `AIMessage` as we can see by printing out the full response without using the print function. Observe that if we do not use the formatting of the `print` function you will see linebreak characters such as `\n` appearing in the message.

In [16]:
response

AIMessage(content='Patricia Industries is a Swedish investment firm that is part of the Investor AB group, one of the largest industrial holding companies in Northern Europe. It was founded in 2006 and is headquartered in Stockholm, Sweden. \n\nPatricia Industries primarily focuses on long-term investments in companies with strong market positions and potential for sustainable value creation. They invest in a wide range of sectors, including healthcare, industrials, technology, and consumer goods. \n\nThe firm aims to support the growth and development of its portfolio companies by providing strategic guidance, operational expertise, and financial resources. They take an active ownership approach and work closely with the management teams of their investments to drive long-term value.\n\nSome notable investments by Patricia Industries include Mölnlycke Health Care, Permobil, Aleris, Grand Group, and Laborie. They have a global presence and invest in companies both in Sweden and interna

### Giving ChatGPT a short-term memory

We can incorporate the `AIMessage` response in a new series of messages and ask a follow up question if you wish. This provides the illusion of the chat model having a memory and being able to continue a conversation for a few turns, as you have seen while working in the browser environment.

In [17]:
# Let's ask a follow-up question which refers to the previous conversation without explicitly mentioning e.g., Patricia Industries
follow_up_question = "Can you say something more about the investment approach?"

# We can now add the response we got from the first query together with our follow-up question
# In python, you can add lists together to form a new list which makes it easy to add new conversation turns into the messages variable
# Not that we add both the chat bot response and our follow up question
messages = messages + [response, HumanMessage(content=follow_up_question)]
messages

[SystemMessage(content='You are a helpful assistant.'),
 HumanMessage(content='What do you know about Patricia Industries from Sweden?'),
 AIMessage(content='Patricia Industries is a Swedish investment firm that is part of the Investor AB group, one of the largest industrial holding companies in Northern Europe. It was founded in 2006 and is headquartered in Stockholm, Sweden. \n\nPatricia Industries primarily focuses on long-term investments in companies with strong market positions and potential for sustainable value creation. They invest in a wide range of sectors, including healthcare, industrials, technology, and consumer goods. \n\nThe firm aims to support the growth and development of its portfolio companies by providing strategic guidance, operational expertise, and financial resources. They take an active ownership approach and work closely with the management teams of their investments to drive long-term value.\n\nSome notable investments by Patricia Industries include Mölnly

In [18]:
# Here we collect the ouput from the chat model in the variable response
response = chat_model(messages)

# We can print out the response by calling on its content using a .content
print(response.content)

Certainly! Patricia Industries follows a structured and disciplined investment approach. Here are some key aspects of their investment strategy:

1. Long-term perspective: Patricia Industries takes a patient and long-term view of their investments. They aim to build sustainable businesses and create value over an extended period rather than seeking short-term gains.

2. Focus on market leaders: They look for companies that have a strong market position and a competitive advantage in their respective industries. Investing in market leaders helps to mitigate risks and provides a solid foundation for long-term growth.

3. Active ownership: Patricia Industries takes an active ownership approach, meaning they actively engage with the management teams of their portfolio companies. They provide strategic guidance, operational support, and access to their network of industry experts to help drive growth and enhance performance.

4. Responsible investing: Sustainability is a key consideration f

By continuing in this fashion we can record a conversation history which we send to the chat model as long as the total text content does not exceed the models content limit.

### OPTIONAL: Using the OpenAI API directly instead of the `langchain` wrapper

The below code snippet calls the OpenAI API directly, which allows us to access **all** of the available input and output options. While this approach offers a higher degree of control, it may not be as easy to use or straightforward to interpret as the functions from langchain.

In [19]:
# Needs to be set for the OpenAI API to be callable
openai.api_key = os.getenv("OPENAI_API_KEY")


# Model and parameters (almost all that are available, logit_bias not included). Parameters model and messages are required
model = "gpt-3.5-turbo-0613"
max_tokens = 1024           # Max nr of tokens to generate in the chat completion, limited by model choice
temperature = 0.7           # Value in (0, 2). Sampling temperature for stochastic nature in response
top_p = 1                   # Vale in (0, 1). Nucleus sampling, optional to temperature, considers tokens comprising top_p probability mass
frequency_penalty = 0       # Value in (-2, 2). Positive value penalizes new tokens based on frequency in text so far, to decrease likelihood of repeating sentences
presence_penalty = 0        # Value in (-2, 2). Positive value penalizes new tokens if appeared in text so far, to increase likelihood of switching to new topics
n=1                         # How many completion options to generate for each message
stream = False              # Boolean value which can allow for streaming response

# More advanced options which allow to include function calling capability for the model itself. We will not use this functionality in the exercises below
function_call = "none"      # 'auto' if call function or generate message, supply {"name": my_func}
functions = [{"name": "Name", "description": "semantic description of what the function(s) do", "parameters": {"type": "object", "properties": {}}}]


# Messages are provided as a list, similarly to the langchain method, but formatted differently
# Other possible values for the key 'role' are 'assistant' and 'function'
system_prompt = "You are a helpful assistant."
user_prompt = "What do you know about Patricia Industries from Sweden?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt},
    ]


# API call, collected in the variable response
response = openai.ChatCompletion.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens,
            top_p=top_p,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty,
            n=n,
            stream=stream,
            function_call=function_call,
            functions=functions
        )


# Printing the response requires a slighty less intuitive format (see below for full unformatted output)
print(response.choices[0].message["content"])

Patricia Industries is a Swedish investment firm specializing in long-term investments in companies across various industries. It is a part of Investor AB, one of the largest investment companies in Northern Europe. Patricia Industries focuses on investing in companies with strong growth potential and a sustainable business model. They aim to be an active and supportive owner, working closely with the management teams of the companies they invest in. Patricia Industries has a diverse portfolio, which includes companies in sectors such as healthcare, technology, and industrial solutions.


We can take a look at the complete response without simplifying or changing the output in any way. When you print the information from the code below, you will see that the response includes details about token usage and other aspects. These details are important as they help you estimate and collect the cost of running queries.

In [20]:
# Printing the response variable without formatting via the print() function displays the full output from the model
response

<OpenAIObject chat.completion id=chatcmpl-8IHYmw1IZflVnHOLzhCGX370nACcS at 0x7f5b85ddf2e0> JSON: {
  "id": "chatcmpl-8IHYmw1IZflVnHOLzhCGX370nACcS",
  "object": "chat.completion",
  "created": 1699368096,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Patricia Industries is a Swedish investment firm specializing in long-term investments in companies across various industries. It is a part of Investor AB, one of the largest investment companies in Northern Europe. Patricia Industries focuses on investing in companies with strong growth potential and a sustainable business model. They aim to be an active and supportive owner, working closely with the management teams of the companies they invest in. Patricia Industries has a diverse portfolio, which includes companies in sectors such as healthcare, technology, and industrial solutions."
      },
      "finish_reason": "stop"
    }
  ],
  "usage":

## Connecting Language Models to the Internet: A way to Expand the models Knowledge base

We will explore two simple open source options for adding internet content as context for our query to the LLM:

* The Wikipedia API which allow us to read in relevant Wikipedia information to get factual information on a specific topic
* The DuckDuckGo API which allows us to search updated information on a topic, to use as context for our query

### Wikipedia API

If we wish to look up or summarize information from Wikipedia and we know the title of the page we're interested in, we can easily use the Wikipedia API to grab the content from that page. However, it's important to know that the Wikipedia API isn't designed for broad searches like a regular search engine. It works best when the search term you enter matches a Wikipedia page title quite closely.

In [21]:
# Let's try it out by asking a general question (note that this is not the recommended way to interact with the Wikipedia API)
query = "What is Patricia Industries"

# Try searching with a general question.
# Here, we are calling a function named wiki_search_api with two arguments; query and how_many
# The how_many argument specifies how many letters/symbols we wish to fetch
result_wiki = wiki_search_api(query, how_many=1000)

# We print the result to the console. This will display the first 1000 characters of the Wikipedia page related to our query
print(result_wiki)

https://en.wikipedia.org/wiki/Patricia_Arquette
Patricia Arquette (born April 8, 1968) is an American actress. She made her feature film debut as Kristen Parker in A Nightmare on Elm Street 3: Dream Warriors (1987) and has since starred in several film and television productions. She has received several awards including an Academy Award, two Primetime Emmy Awards, and three Golden Globe Awards.
She had starring roles in several critically acclaimed films, including True Romance (1993), Ed Wood (1994), Flirting with Disaster (1996), Lost Highway (1997), The Hi-Lo Country (1998), and Bringing Out the Dead (1999). From 2005 to 2011, she starred as a character based on the medium Allison DuBois in the supernatural drama series Medium, winning the Primetime Emmy Award for Outstanding Lead Actress in a Drama Series in 2005.
For playing a single mother in the coming-of-age film Boyhood (2014), which was filmed from 2002 until 2014, Arquette won the Academy Award for Best Supporting Actress. 

As we see the answer is not related to Patricia Industries at all. Instead the response is from a popular Wikipedia page with a Title trying to match our query.


By searching instead for a keyword we can increase the chances of obtaining a more relevant answer.

In [22]:
# This is the keyword you want to search for on Wikipedia.
query_keyword = "Patricia Industries"

# By searching on the keyword we obtain a better match than if asking a general question.
result_wiki = wiki_search_api(query_keyword, how_many=1000)
print(result_wiki)

https://en.wikipedia.org/wiki/Investor_AB
Investor AB is a Swedish investment and holding company, often considered a de facto conglomerate. One of Sweden's largest companies, Investor AB serves as the investment arm of the prominent Swedish Wallenberg family; the family's companies are involved in a variety of industries, of which the primary industries are pharmaceuticals, telecommunications and industry.Investor AB is Sweden's most valuable publicly traded company; it has major or controlling holdings in several of Sweden's other largest companies. It has major investments worldwide through Patricia Industries and EQT Partners.


== History ==
Investor AB was established in Stockholm when new Swedish legislation made it more difficult for banks to own stocks in industrial companies on a long-term basis. The shareholdings of the Wallenberg family bank, Stockholms Enskilda Bank, were transferred to Investor AB, a newly formed industrial holding company spun off from the bank, which wo

We can customize this somewhat by adding a function which extracts the keyword from a sentence by looking for the phrase `keyword: <KEYWORD>`. This has some limitiations though since it requires us to include this kind of formatted phrase in our query.

In [23]:
query_keyword = "Patricia Industries"
query_with_keyword = f"What is keyword: {query_keyword}"

# By searching on the keyword we obtain a better match.
# The find_keyword() function below extracts the keyword, so this is doing the same as we did above by supplying the keyword directly
result_wiki = wiki_search_api(find_keyword(query_with_keyword), how_many=1000)
print(result_wiki)

https://en.wikipedia.org/wiki/Investor_AB
Investor AB is a Swedish investment and holding company, often considered a de facto conglomerate. One of Sweden's largest companies, Investor AB serves as the investment arm of the prominent Swedish Wallenberg family; the family's companies are involved in a variety of industries, of which the primary industries are pharmaceuticals, telecommunications and industry.Investor AB is Sweden's most valuable publicly traded company; it has major or controlling holdings in several of Sweden's other largest companies. It has major investments worldwide through Patricia Industries and EQT Partners.


== History ==
Investor AB was established in Stockholm when new Swedish legislation made it more difficult for banks to own stocks in industrial companies on a long-term basis. The shareholdings of the Wallenberg family bank, Stockholms Enskilda Bank, were transferred to Investor AB, a newly formed industrial holding company spun off from the bank, which wo

**Try it out yourself:**

Try it out by searching with your own keyword. The more closely this matches an existing Wikipedia title page, the more likely you are of finding what you look for.

#### Explicitly using the query for the purpose of answering the question

In [24]:
# Step 1:
# Choose a keyword for your search. It's better to pick a keyword that closely matches an existing Wikipedia page title.
# This increases your chances of finding the information you're looking for.
# If you can't come up with something try query_keyword = "rubberducking"
query_keyword = "..."

# Step 2:
# Now, let's search Wikipedia using the keyword you chose.
# The 'how_many' argument tells the program how many characters of the result you want to retrieve.
# In this case, we've set it to 1000, so you'll see the first 1000 characters of the Wikipedia page.
result_wiki = wiki_search_api(query_keyword, how_many=1000)

# Step 3:
# Print the result to the screen so that you can read it.
print(result_wiki)

https://en.wikipedia.org/wiki/Ellipsis
The ellipsis ... (; also known informally as dot dot dot) is a series of dots that indicates an intentional omission of a word, sentence, or whole section from a text without altering its original meaning. The plural is ellipses. The term originates from the Ancient Greek: ἔλλειψις, élleipsis meaning 'leave out'.Opinions differ as to how to render ellipses in printed material. According to The Chicago Manual of Style, it should consist of three periods, each separated from its neighbor by a non-breaking space: . . .. According to the AP Stylebook, the periods should be rendered with no space between them: .... A third option is to use the Unicode character U+2026 … HORIZONTAL ELLIPSIS.


== Background ==
The ellipsis is also called a suspension point, points of ellipsis, periods of ellipsis, or (colloquially) "dot-dot-dot". Depending on their context and placement in a sentence, ellipses can indicate an unfinished thought, a leading statement, a s

#### Explicitly using Wikipedia facts for the purpose of answering a question

In [25]:
# Step 1:
# Setting up the Model
# We're preparing to use a specific AI model by setting some initial parameters.
# One of these parameters is 'temperature', which affects how creative or strict the model's responses will be.
# A lower temperature like 0.0 makes the model more focused and less random in its responses.
temperature = 0.0


# Now we create an instance of the ChatOpenAI model, which we'll use to generate responses.
# We're specifying which model to use ('gpt-3.5-turbo') and providing our OpenAI API key to authorize access.
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model="gpt-3.5-turbo",
    temperature=temperature
)


# Step 2:
# Defining a Prompt Function to Use the Model
# We're creating a function to make it easier to use the model with different questions and keywords.
# This function will search Wikipedia for context, then ask the model to answer a query based on that context.
def LLM_with_wiki_prompt(query: str, query_keyword: str, how_many: int=2000, temperature: float=0.0):
    '''
    Prompt function for calling the chat model with a general query, where we believe relevant information can
    be found on a Wikipedia page whose title matches the query_keyword.
    We an optionally change how many characters should be retrieved and what the temperature of the model should be.
    '''

    # Retrieve serch results based the query
    # This line searches Wikipedia using the keyword provided, and gets up to 2000 characters of text as context.
    context = wiki_search_api(query_keyword, how_many=how_many)

    # System prompt
    # We're setting up a system prompt to instruct the model on how to approach answering the query.
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful investment management expert.
    You are logical, methodical and always find the best and most relevant answer to a query.
    Break down the problem, objects, numbers and logic before starting to answer the query.
    Then proceed to answer in a step-by-step manner.
    """

    # This prompt inputs the query and telling the model to consider the Wikipedia context we found.
    user_prompt = f"""
    Consider carefully the following wiki facts as context.
    context: {context}

    Use only the above context and nothing else to answer the following query and summarize your response.
    query: {query}
    If the answer is provided in the form of a bulleted list within the context,
    then return those bullets verbatim and do not try to rephrase them.
    """

    # We're arranging the system and user prompts into a list of messages to send to the model.
    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]


    # We ask the model to generate a response based on the messages, and print that response.
    response = chat_model(messages, temperature=temperature)
    print(response.content)

In [26]:
# We can now ask a general question as our query
query = "What is the relationship between Patricia Industries and Investor AB?"
# Together with a keyword argument which should match a Wikipedia title where relevant information can be found
query_keyword = "Patricia Industries"

# Using the above query and keyword as arguments, we call the prompt function
LLM_with_wiki_prompt(query=query, query_keyword=query_keyword, temperature=temperature)

https://en.wikipedia.org/wiki/Investor_AB
- Investor AB has major investments worldwide through Patricia Industries.
- Patricia Industries is a subsidiary or division of Investor AB.
- Patricia Industries is the investment arm of Investor AB for its global investments.
- Patricia Industries is involved in various industries, including pharmaceuticals, telecommunications, and industry, just like Investor AB.
- Patricia Industries is responsible for managing and overseeing Investor AB's major investments outside of Sweden.


**Try it out yourself:**

Try it out by searching with your own query and relevant keyword. The more closely the keyword matches an existing Wikipedia title page, the more likely it is that the LLM will be able to answer your query based on relevant Wikipedia facts.

In [27]:
# Assign the query string, which is the question you want to answer.
query = "..."

# Assign the query "keyword" string, which is the keyword you want to use to fetch relevant context from Wikipedia.
query_keyword = "..."


# Now, you are calling the function LLM_with_wiki_prompt which was defined earlier.
# You are passing the query, query_keyword, and temperature (defined earlier) as arguments to the function.
# This function is expected to search Wikipedia using the query_keyword,
# then use the model to answer the query based on the Wikipedia information,
# and finally the function prints out the model's response.
LLM_with_wiki_prompt(query=query, query_keyword=query_keyword, temperature=temperature)

https://en.wikipedia.org/wiki/Ellipsis
The ellipsis, also known as dot dot dot, is a series of dots that indicates an intentional omission of a word, sentence, or whole section from a text without altering its original meaning. It can be rendered in different ways, such as three periods with non-breaking spaces between them, three periods with no space between them, or the Unicode character U+2026. Ellipses can be used to indicate an unfinished thought, a leading statement, a pause, an echoing voice, or a silence. They can also suggest melancholy or longing when placed at the end of a sentence. Style guides have their own rules for the use of ellipses, with variations in spacing and punctuation. The question "..." is not clear and requires further information to provide a specific answer.


### DuckDuckGo search API

The [DuckDuckGo browser](https://duckduckgo.com/) provides an open source API which can be used to browse and search the internet. This returns a search response to a query and can be useful for incorporating a small amount of updated information as context to your LLM queries. More advanced search functionalities are possible but quickly become more technically involved for the user, so for the purpose of this short course we restrict to the basic search functionality.

In [28]:
# Let's pose a query for which we wish to retrieve information from the internet
query = "What's the latest business news about Investor AB?"

# Note that the retrieved responses are not necessarily the same every time we run a search.
# The max_results argument is a cap of the nr of retrieved search results
response = get_search_results_ddg(query, max_results=50)
print(response)

November 2, 2023 at 7:06 AM · 3 min read An In-depth Look into Investor AB's Dividend Performance and Sustainability Investor AB ( IVSBF) recently announced a dividend of $1.1 per share,... Investor AB INVEb.ST Latest Trade 214.25 SEK -3.1 -1.43% As of Oct 18, 2023. Values delayed up to 15 minutes Today's Range 214.00 - 218.15 52 Week Range 163.08 - 223.60 Profile Charts... Investor AB : Financial news and information Stock Investor AB | Nasdaq Stockholm: INVE B | Nasdaq Stockholm ... Business Leaders; Sectors ... CEO Johan Forssell will leave Investor in May 2024 in a new role with focus on Oct. 20. AQ Johan Forssell to Leave from Investor AB as President and Director, Effective 7 May 2024 ... Investor AB + Add to watchlist INVE B:STO Actions Financials Investment Banking and Brokerage Services Price (SEK) 204.40 Today's Change 1.95 / 0.96% Shares traded 2.43m 1 Year change +13.33%... Today Investor AB publishes the Annual Report 2022 including the Sustainability Report, on www.invest

Note that you receive a collection of relevant snippets to your query and not an actual answer. These snippets correspond to small amount of information retrieved from various web pages based om the query. To collect the retrieved information into a more summarized answer we can use the LLLM.

#### Explicitly using the query for the purpose of answering a question

Now we are ready to implement the internet search functionality together with the LLM for answering our queries.

In [29]:
# Step 1:
# Setting up the Model
# We're preparing to use a specific AI model by setting some initial parameters.
# One of these parameters is 'temperature', which affects how creative or strict the model's responses will be.
# A lower temperature like 0.0 makes the model more focused and less random in its responses.
temperature=0.0


# Now we create an instance of the ChatOpenAI model, which we'll use to generate responses.
# We're specifying which model to use ('gpt-3.5-turbo') and providing our OpenAI API key to authorize access.
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model="gpt-3.5-turbo",
    temperature=temperature
)


# Step 2:
# Defining a Prompt Function to Use the Model
# We define a function for more easily calling the LLM using the same prompt template but with a different query
# The function uses DuckDuckGo to retrieve context based on the query and then use that to get a response from the LLM
def LLM_with_search_prompt(query: str, max_results: int=10, temperature: float=0.0):

    # Retrieve serch results based the query
    context = get_search_results_ddg(query, max_results=max_results)

    # System prompt
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful investment management expert.
    You are logical, methodical and always find the best and most relevant answer to a query.
    Break down the problem, objects, numbers and logic before starting to answer the query.
    Then proceed to answer in a step-by-step manner.
    """

    # This prompt inputs the query and context
    user_prompt = f"""
    One of our portfolio companies have handed us these news reports as context.
    context: {context}

    Use only the above context and nothing else to answer the following query and summarize your response.
    query: {query}
    If the answer is provided in the form of a bulleted list within the context,
    then return those bullets verbatim and do not try to rephrase them.
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]

    response = chat_model(messages, temperature=temperature)
    print(response.content)

Running the above function will print out the response. Note that running the below cell may result in different outputs despite setting the temperature parameter to 0. This is because the retrieved answers from the DuckDuckGo search are not always the same, which means that the LLM may consider different contexts when trying to answer your query.

In [30]:
# Let's pose a query for which we wish to retrieve information from internet and get an LLM response based on that information
query = "What's the latest business news about Investor AB and its board?"

# Here we use our prompt function to call the LLM
LLM_with_search_prompt(query=query, temperature=temperature)

- CEO Johan Forssell will leave Investor AB in May 2024 in a new role with a focus on Oct. 20.
- AQ Johan Forssell to Leave from Investor AB as President and Director, Effective 7 May 2024.


**Try it out yourself:**

Try it out by searching with your own query. The context will be automatically included via the prompt function defined above.

In [31]:
# Write your own query and retrieve search results from that as context for the LLM
query = "What is rubberducking?"

LLM_with_search_prompt(query=query, temperature=temperature)

- Rubberducking is a debugging tool used by programmers.
- It involves explaining the code and the problem to a rubber duck.
- The process helps programmers identify and solve issues in their code.
- Rubberducking can be done by speaking or writing about the problem in natural language.
- It is also known as rubber duck debugging or rubberducking.
- Some programmers keep a rubber duck on their desk for debugging emergencies.
- There is a game called Jeep Ducking where rubber ducks are placed on other people's Jeep vehicles to make them smile.


## PDF knowledge base

We start by providing the location and names of the PDF files which we will be exploring.

In [32]:
# Absolute or relative path to folder where the PDF files are located
DATAPATH = "/content/drive/MyDrive/GenAI/GenAI short course/"



# Name of the PDF files (without the trailing .pdf file endings)
Patricia_permobil_manual = "Patricia_permobil_manual"
caterpillar_10k = "Caterpillar-10k"
whirlpool_10k = "whirlpool-10k"
electrolux_ann_rep = "Electrolux-annual-report"


# We collect all the PDF file names in a list
pdf_files = [
    Patricia_permobil_manual,
    caterpillar_10k,
    whirlpool_10k,
    electrolux_ann_rep
]

> **We load and inspect the text converted PDF files and some properties below**

We can now load one of the PDFs and examine its content. Note that this relies on extracting the content of the PDF as text and works well for the parts which are written text in the PDF, but works less well for e.g., tables and figures. In order to handle tables and figures well we typically need to perform pre-processing steps which are more complex and tailored for the specific PDF file type we wish to examine.

In [33]:
pdf_file = Patricia_permobil_manual

# Here we read in the PDF as text using the function text_from_pdf(), defined in the file helper_functions.py
pdf_text = text_from_pdf(DATAPATH + pdf_file + ".pdf")


# Here we count and print out the total nr of pages in the retrieved PDF
print(f"{Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}Nr of pages in pdf document:{Style.RESET_ALL} {len(pdf_text.keys())} \n")


# Here we count and print out the total nr of symbols in the retrieved PDF file
concat_text = pdf_dict_to_str(text_from_pdf(DATAPATH + pdf_file + ".pdf"))
print(f"The document contains {Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}{len(concat_text)}{Style.RESET_ALL} symbols/characters")


# We then print an example page from the PDF
print(f"{Fore.RED + Back.LIGHTYELLOW_EX + Style.BRIGHT}\n\nEx text from first page:{Style.RESET_ALL}\n")
print(f"{pdf_text['page_3']}")

Nr of pages in pdf document: 96 

The document contains 195303 symbols/characters


Ex text from first page:

ZR Owner’s Manual
OM0005_Rev A_ZR
ii
REGISTER YOUR TiLITE
Register online at TiLite.com
or
Complete and mail the form on the next page
Why Should You Register:
1. Increase your use and enjoyment of your TiLite by receiving updates from
    TiLite with product information, maintenance tips and industry news.
2. Enable TiLite to contact you or your health care provider if servicing is
    needed for your wheelchair.
3. Provide your feedback to TiLite regarding your experience and needs,
    thereby enabling TiLite to further improve product designs.
All information you provide to TiLite when you register will be protected by TiLite as 
required by applicable laws and regulations and will be used solely by TiLite.



### OPTIONAL: The following section performs chunking of text and creates indexed embeddings for the chunks

The chat models have a limit in the number of tokens/characters they can receive as input. This means we cannot feed them the full PDF as context. Rather we need to find the most relevant pieces of text within the PDF and use that when asking the LLM to respond to a query about the PDF.

Here we split up the four PDF documents into piecewise chunks of text for embedding purposes.

In [34]:
chunked_pdf_files_dict = {}
for pdf_file in pdf_files:
    pdf_location = DATAPATH + pdf_file + ".pdf"
    chunked_pdf = chunk_documents(
        documents= TextLoader(file_path=pdf_location, loader=PyPDFLoader),
        TextSplitter=RecursiveCharacterTextSplitter,
        chunk_size=512, # 512 is the maximum Sequence length
        chunk_overlap=20,
        separator=None,
        )
    chunked_pdf_files_dict[pdf_file] = chunked_pdf

Let's print out a sample chunk so that we can see what kind of information is available within a single chunk of text.

In [35]:
print(chunked_pdf_files_dict[Patricia_permobil_manual][79].page_content)

3. The assistant at the rear of the chair is in control of this procedure. He or she must tilt the chair back to its balance point on the rear wheels. NEVER attempt to lift a wheelchair by lifting on any removable (detachable) parts, including upholstery and removable push handles or push handle grips. 
4. The second assistant at the front must firmly grasp a non-detachable part of the front frame (but NOT swing away hangers) with both hands and lift the chair up and over one stair at a time.


We can also inspect a chunk which contains a table, to see what it looks like.

In [36]:
# The below chunk contains a table from the caterpillar_10k PDF.
print(chunked_pdf_files_dict[caterpillar_10k][104].page_content)

agreement.  At select business units, we have hired certain highly specialized employees under employment contracts that specify a
term of employment, pay and other benefits.
 
Full-T ime Employees at Year-End
 2022 2021
Inside U.S. 48,200 44,300
Outside U.S. 60,900 63,400
Total 109,100 107,700
By Region:   
North America 48,700 44,700
EAME 16,900 17,600
Latin America 19,100 19,500
Asia/Pacific 24,400 25,900
Total 109,100 107,700


#### Performing the embedding of the chunked PDF documents

In [37]:
embed_pdfs = False
save_embeddings = False

# For embeddings we use the nr 1 model on the MTEB leaderboard at https://huggingface.co/spaces/mteb/leaderboard
embedding_model = ["BAAI/bge-large-en-v1.5"]

> **NB: The below code block takes around 100 minutes to complete for the four PDF's (100-150 pages each), so this has been prepared before the lecture. We have kept the code here for those interested, but `DO NOT RUN THE BELOW TWO CELLS!` or `DO NOT MODIFY THE embed_pdfs = False and save_embeddings = False!`**

In [38]:
if embed_pdfs:

  embedding_dict = {}
  for model_name in embedding_model:
    for pdf_file in pdf_files:
      print(f"Creating FAISS embedding for document {pdf_file} using - {model_name}")
      start = timer()
      embedding = doc_embedding(embedding_model=model_name)
      faiss_index = make_index_FAISS(chunked_documents=chunked_pdf_files_dict[pdf_file], embedding=embedding)
      end = timer()
      time_taken=end-start
      print(f"Done! Embedded vector index created after {time_taken/60:.2f} minutes")
      print("*"*25)
      embedding_dict[pdf_file] = faiss_index

Here we save the embeddings so that we don't have to rerun the above every time.

In [39]:
if save_embeddings:
    for pdf_embedding in embedding_dict.keys():
        FAISS_folder_name = DATAPATH + f"FAISS_{pdf_embedding}.faiss"
        embedding_dict[pdf_embedding].save_local(FAISS_folder_name)

Note that the above two code blocks have been run in advance in order to save the embeddings. This allows us to avoid doing this during the live session.

### Loading the saved embeddings

If embeddings are already saved we can simply load them from storage. This avoids repeating the embedding process which can be quite time consuming. In a production environment you typically do this kind of indexing once, or on a running schedule if you expect the documents you embed will change over time.

In [40]:
# We load and collect the saved embeddings in a python dictionary. This only takes ~15 seconds to load
faiss_embeddings_dict = {}
if not save_embeddings:
    for pdf_file in pdf_files:
        FAISS_folder_name = DATAPATH + f"FAISS_{pdf_file}.faiss"
        embedding = doc_embedding(embedding_model=embedding_model[0])
        faiss_embeddings_dict[pdf_file] = FAISS.load_local(FAISS_folder_name, embedding)

Let's try out making a simple query against the saved embeddings by doing a similarity search.

In [41]:
# We choose one of the PDF files and a relevant search query
pdf_file = Patricia_permobil_manual
similarity_query = "What is the procedure for setting up my TiLite rigid wheelchair?"


# We retrieve the most semantically similar chunks by doing a similarity search over the saved embeddings in our faiss_embeddings_dict
# The argument k sets how many such chunks we wish to retrieve
test = faiss_embeddings_dict[pdf_file].similarity_search(similarity_query, k=10)


# We can now print out the most similar chunk. We print out both the chunk and the PDF page on which it is located.
# The retrieved chunks are collected in order of similarity from 0 to k-1,
# so chunk_nr = 0 correspond to the most similar chunk
chunk_nr = 0
print(f"Found on page: {test[chunk_nr].metadata['page']}\n")
print(test[chunk_nr].page_content)

Found on page: 78

11-4
ZR Owner’s Manual OM0005_Rev A_ZRCHAPTER 11: CASTERS AND FORKSNote: TiLite designs its rigid wheelchairs to be flexible for improved maneuverability and increased ride comfort.  
However, this flexibility requires that your chair be set up properly.  The following procedure will enable you to set up  your TiLite rigid wheelchair so it will perform to its potential.
1. Place the wheelchair on a smooth, level surface with the casters trailing rearward.


### Querying the Permobil Manual PDF

We are now ready to start asking questions of our PDF files.

In [42]:
# Step 1:
# Setting up the Model
# We're preparing to use a specific AI model by setting some initial parameters.
# One of these parameters is 'temperature', which affects how creative or strict the model's responses will be.
# A lower temperature like 0.0 makes the model more focused and less random in its responses.
temperature = 0.0


# Now we create an instance of the ChatOpenAI model, which we'll use to generate responses.
# We're specifying which model to use ('gpt-3.5-turbo') and providing our OpenAI API key to authorize access.
chat_model = ChatOpenAI(
    openai_api_key=os.environ['OPENAI_API_KEY'],
    model="gpt-3.5-turbo",
    temperature=temperature
)


# Step 2:
# Defining a Prompt Function to Use the Model
# We define a function for more easily calling the LLM using the same prompt template but with a different query and context
def LLM_with_pdf_context(context: str, query: str, temperature: float=0.0):

    # System prompt
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful investment management expert.
    You are logical, methodical and always find the best and most relevant answer to a query.
    Break down the problem, objects, numbers and logic before starting to answer the query.
    Then proceed to answer in a step-by-step manner.
    """

    # This prompt template inputs the merged context and query for the LLM
    user_prompt = f"""
    I need help from an investement management expert.
    One of our portfolio companies have handed us these brief instruction snippets as
    context: {context}

    Use only the above context and nothing else to answer the following
    question: {query}
    If the answer is provided in the form of a bulleted list within the context,
    then return those instructions verbatim and do not try to rephrase them.
    If you cannot answer based on the information in context, then do not try to answer, but instead must answer verbatim with:
    {{There is no available information related to your query in the context!'}}
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]

    response = chat_model(messages, temperature=temperature)
    print(response.content)

In [43]:
# We choose one of the PDF files and a relevant search query. This query is the same as we asked during session 1 using the AskYourPDF plugin
pdf_file = Patricia_permobil_manual
query = "What tools are needed for adjusting the Front Seat Height on Slipstream Single-Sided Forks. Also provide the steps for performing the adjustment."


# We retrieve the most similar chunks related to above search query and collect these into a context string
faiss_index = faiss_embeddings_dict[pdf_file]
top_hits = similarity_search_FAISS(search_query=query, nr_hits=5, index_store=faiss_index)
context = "".join([document.page_content for document in top_hits])

First, let's use all of the retrieved top hits as context when querying the model.

In [44]:
# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

Based on the information provided in the context, the tools needed for adjusting the Front Seat Height on Slipstream Single-Sided Forks are:

- 5/8" Open End Wrench
- Screwdriver

The steps for performing the adjustment are as follows:

1. Remove the caster. Refer to "Slipstream Single-Sided Forks - Replacing Casters" on page 11-2 for instructions on how to remove the caster.
2. Follow the procedures under "Standard Forks - Replacing Casters" on page 11-1 to mount the casters in the alternative axle holes in the fork. This will allow you to adjust the Front Seat Height without changing the casters to a larger or smaller size.
3. Note that the full range of adjustability may not be available with 5" or 6" casters. Additional adjustability may be achieved with different forks or casters or with fork stem extensions.

Please note that the instructions provided above are verbatim from the context.


We see that the instructions are incomplete (these contain 7 steps in the PDF). This is due to the fact that the most relevant chunk happened to only contain the first part of the full instructions. Since the chunks are small this is something that can happen quite often. In order to get a more complete response we can try to feed not the retrieved and merged chunks as context, but instead feed the whole PDF page of the most relevant chunk, assuming it contains more fuller context. We can also do some combination of this and using other chunks if we find that top hit doesn't always work.

In [45]:
# Here we get the page of the top hit and use that whole PDF page as context
context_page = f"""{pdf_text[f'page_{top_hits[0].metadata["page"]}']}"""

# Use the pre-defined function but with the whole context page as context
LLM_with_pdf_context(context_page, query)

Based on the provided context, the tools needed for adjusting the Front Seat Height on Slipstream Single-Sided Forks are:

- 5/8" Open End Wrench
- Screwdriver

The steps for performing the adjustment are as follows:

1. Remove the caster. Refer to "Slipstream Single-Sided Forks - Replacing Casters" on page 11-2 for instructions.
2. Using the shaft of the screwdriver, remove the E-Ring by pressing downward across the open portion of the E-Ring. Ensure you wear protective eyewear. See Figure 11-3.
3. Using the 5/8" Open End wrench, remove the axle from the Slipstream Single-Sided Fork.
4. Place the axle in the alternate axle hole and securely tighten.
5. Using the shaft of the screwdriver, replace the E-Ring by pressing downward across the closed portion of the E-Ring, snapping the E-ring into place. See Figure 11-3.
6. Replace the caster. Refer to "Slipstream Single-Sided Forks - Replacing Casters" on page 11-2 for instructions.
7. Follow Steps 1 through 6 on the opposite fork.

Please

In this case this worked much better, since the retrieved page actually contained the full instructions. In real production applications you may not always be so lucky, the instructions could for example be spread over multiple pages, so some work is often required to ensure consistency over a large corpus of texts.

### Querying the Electrolux Annual Report PDF

Let's try asking some questions based on the content in the Electrolux's annual report for 2022.

In [46]:
# Set our pdf variable to the Electrolux annual report
pdf_file = electrolux_ann_rep

In [47]:
query = "Which major events happened for Electrolux during 2022?"


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

- January 11: Electrolux announced a loss for the fourth quarter of 2022, with an estimated operating income of approximately SEK -2.0bn (0.9), including non-recurring items of SEK -1.4bn (-0.7).
- February 1: Electrolux decided to discontinue production at the Nyíregyháza factory in Hungary from the beginning of 2024.
- September 9: Electrolux divested its business in Russia and sold its Russian subsidiary to local management, recording a capital loss of SEK 350m.
- April 29 to September 2: Electrolux completed a share buyback program, repurchasing a total of 17,369,172 own series B shares for a total amount of SEK 3,032m.


**Try it out yourself:**

Try it out by searching with your own query for information from the Electrolux Annual Report. You could, for example, ask questions about risks and challenges mentioned in the report or ask the model to explain Electrolux's business in Latin America.

In [48]:
# Write your own query and retrieve search results from that as context for the LLM
query = "..."


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

{There is no available information related to your query in the context!}


### Querying the Caterpillar 10-K PDF

Let's try asking some questions based on the content in the Caterpillar 10-K annual report for 2022.

In [49]:
# Set our pdf variable to the Caterpillar 10-K report
pdf_file = caterpillar_10k

In [50]:
# Ask a relevant query
query = "Which identified risks and challenges are mentioned in the report?"


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

The identified risks and challenges mentioned in the report are:

- Delays and increased costs resulting from supply chain challenges and availability issues with suppliers.
- Freight delays that could impact production in the company's facilities.
- Fluctuations in freight costs, fuel costs, and limitations on shipping and receiving capacity.
- Disruptions in the transportation and shipping infrastructure at important geographic points of exit and entry for the company's products.
- Operating in different regions and countries exposes the company to numerous risks, including multiple and potentially conflicting laws, regulations, and policies that are subject to change.

Please note that the information provided above is a direct excerpt from the context and is presented in a bulleted list format.


**Tables in PDF-reports:**

The Caterpillar 10-K report contains multiple tables which may cause a problem when using the text chunks for similarity retrieval. Here we can try and ask questions about information found in tables in the report. However, depending on the chunks, the model may confuse numbers found in the tables.

In [51]:
# Let's ask a question which requires analyzing table information inside the PDF
query = "How many full-time employees did they have at the end of the year 2022 in Latin America?"


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

Based on the provided context, we can find the information about the number of full-time employees in Latin America at the end of the year 2022. 

The relevant information is mentioned in the snippet: "Latin America\nCurrent 770 400 150 69 26 20 — 1,435". 

Therefore, the number of full-time employees in Latin America at the end of the year 2022 is 1,435.


**Try it out yourself:**

Try it out by searching with your own query for information from the Caterpillar 10-K report.

In [52]:
# Write your own query and retrieve search results from that as context for the LLM
query = "..."


# Use our function to retrieve the most similar chunks related to the search query for the given PDF
context = get_contexts(pdf=pdf_file, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Use our pre-defined function to get the model response
LLM_with_pdf_context(context, query)

{There is no available information related to your query in the context!}


### Comparison between two PDF: Electrolux Annual Report and Whirlpool 10-K Report

Let's go a bit deeper and try to compare information between two different PDF files. We can try to ask a question for which we expect there are answers in both the Electrolux annual report and the Whirlpool 10-K report. By using a query to retrieve relevant information from both of these reports we can use that as context to the LLM and ask about comparisons etc.

In [53]:
# We define a function for more easily call the LLM using the same prompt template but with a different query and contexts
def LLM_with_pdf_context_comparison(company1: str, company2: str, context1: str, context2: str, query: str, temperature: float=0.0):

    # System prompt
    system_prompt = f"""
    Ignore all previous instructions. You are a helpful investment management expert.
    You are logical, methodical and always find the best and most relevant answer to a query.
    Break down the problem, objects, numbers and logic before starting to answer the query.
    Then proceed to answer in a step-by-step manner.
    """

    # This prompt inouts the merged context
    user_prompt = f"""
    Two of our portfolio companies have handed us these brief instruction snippets as
    contexts. Contexts for {company1}: {context1}, and for {company2}: {context2}

    Use only the above contexts and nothing else to answer the following
    query: {query}
    If the answer is provided in the form of a bulleted list within the context,
    then return those instructions verbatim and do not try to rephrase them.
    If you cannot answer based on the information in context, then do not try to answer, but instead must answer verbatim with:
    {{There is no available information related to your query in the context!'}}
    """

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_prompt),
    ]

    response = chat_model(messages, temperature=temperature)
    print(response.content)

With the help of the above function we can pose queries which prompts the LLM to compare the information in different PDF's by creating separate contexts. Here, we compare the information in the Electrolux Annual report and Whirlpool 10-K report.

In [54]:
# Let's make a query which we would like to pose to both the Whirlpool and the Electrolux PDF files
query = "Which key business strategies were mentioned in the report?"


# Using the above query, we can fetch relevant contexts from both PDF files
context_whirlpool = get_contexts(pdf=whirlpool_10k, query=query, faiss_embeddings_dict=faiss_embeddings_dict)
context_electrolux = get_contexts(pdf=electrolux_ann_rep, query=query, faiss_embeddings_dict=faiss_embeddings_dict)

In [55]:
# Given the above retrieved contexts we can now ask a comparative question between the companies
chat_query = "These are the key business strategies for Whirlpool and Electrolux. Compare them. Which strategies are similar and which are different?"

In [56]:
# Use our pre-defined function together with the names of the companies and their context
LLM_with_pdf_context_comparison("Whirlpool", "Electrolux", context_whirlpool, context_electrolux, chat_query)

- The key business strategies for Whirlpool include:
  - Value creating approach enabled by three strong pillars: small appliances, major appliances in the Americas and India, and commercial appliances.
  - Commitment to investing in businesses that support high growth and high margins.
  - Growth strategies, financial results, product development, and sales efforts.
  - Facing competition that may be able to quickly adapt to changing consumer preferences, particularly in the connected appliance space, or may be able to adapt more quickly to changes brought about by the global pandemic, supply chain constraints, inflationary pressures, currency fluctuations, geopolitical uncertainty.
- The key business strategies for Electrolux include:
  - Focus on driving sustainable consumer experience innovation and increasing efficiency.
  - Long-term strategy based on five key industry trends.
  - Financial targets for profitable growth over a business cycle.
  - Clear strategy to deliver profita

**Try it out yourself:**

Try it out by telling your model what information from the reports you want to compare, for example, their sustainability strategies.

In [57]:
# Write your own query on what you like to pose to both the Whirlpool and the Electrolux PDF files
query = "..."


# Using the above query, we can fetch relevant contexts from both PDF files
context_whirlpool = get_contexts(pdf=whirlpool_10k, query=query, faiss_embeddings_dict=faiss_embeddings_dict)
context_electrolux = get_contexts(pdf=electrolux_ann_rep, query=query, faiss_embeddings_dict=faiss_embeddings_dict)


# Change this query to ask the model to compare ... for the companies.
chat_query = "This is the ... for Whirlpool and Electrolux. Compare them."


# Use our pre-defined function together with the names of the companies and their context
LLM_with_pdf_context_comparison("Whirlpool", "Electrolux", context_whirlpool, context_electrolux, chat_query)

There is no available information related to your query in the context!


### End of the notebook

> **NB: After completed course we can flush and unmount drive**

In [59]:
# First flush and unmount drive after we are done,
# but to re-mount with new login we may need to remove dir manually first
if use_drive:
  drive.flush_and_unmount()
  !rm -rf /content/drive
#drive.mount('/content/drive', force_remount=True)

Drive not mounted, so nothing to flush and unmount.
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
